In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('/kaggle/input/sentimental-analysis-for-tweets/sentiment_tweets3.csv')
data=data.drop('Index',axis=1)

data

In [ ]:
data['message to examine'][999]

In [ ]:
import re #[\s!-@^#$]
data['message to examine'] = data['message to examine'].apply(lambda x: re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','',str(x)))
data['message to examine'] = data['message to examine'].apply(lambda x: re.sub('[!:*.,-]|[@]','',str(x)))
data['message to examine'] = data['message to examine'].apply(lambda x : x.lower())
data

In [ ]:
data['message to examine'][999]

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

In [ ]:
data['message to examine'] = data['message to examine'].apply(lambda x : word_tokenize(x))
data.head()

In [ ]:
data['message to examine'][999]

In [ ]:
!pip install contractions
from contractions import contractions_dict

In [ ]:
contractions_dict.keys()

In [ ]:
def contracted_word_expansion(token):
    if token in contractions_dict.keys():
        return contractions_dict[token]
    else:
        return token

def contractions_expansion(list_of_tokens):
    return map(contracted_word_expansion,list_of_tokens)

In [ ]:
data['message to examine'] = data['message to examine'].apply(lambda x: contractions_expansion(x))
data['message to examine'] = data['message to examine'].apply(lambda x: list(x))
data.head()

In [ ]:
data['message to examine'][999]

In [ ]:
regex = r'^@[a-zA-z0-9]|^#[a-zA-Z0-9]|\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*|\W+|\d+|<("[^"]*"|\'[^\']*\'|[^\'">])*>|_+|[^\u0000-\u007f]+'
def remove_waste_words(token):
    return re.search(regex, token)
def filter_waste_words(list_of_tokens):
    return filterfalse(remove_waste_words,list_of_tokens)

In [ ]:
from itertools import filterfalse
data['message to examine'] = data['message to examine'].apply(lambda x: filter_waste_words(x))

In [ ]:
data['message to examine'] = data['message to examine'].apply(lambda x: list(x))

In [ ]:
data['message to examine'][999]

In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords, wordnet
from spacy.lang.en.stop_words import STOP_WORDS

In [ ]:
en_stop_words = list(set(stopwords.words('english')).union(set(STOP_WORDS)))

In [ ]:
len(en_stop_words)

In [ ]:
def is_stopword(token):
    return not(token in en_stop_words or re.search(r'\b\w\b|[^\u0000-\u007f]+|_+|\W+',token))

def stopwords_removal(list_of_tokens):
    return filter(is_stopword,list_of_tokens)

In [ ]:
data['message to examine'] = data['message to examine'].apply(lambda x: stopwords_removal(x))
data['message to examine'] = data['message to examine'].apply(lambda x: list(x))

In [ ]:
data['message to examine']

In [ ]:
def get_wnet_pos_tag(treebank_tag):
    if treebank_tag[1].startswith('J'):
        return (treebank_tag[0],wordnet.ADJ)
    elif treebank_tag[1].startswith('V'):
        return (treebank_tag[0],wordnet.VERB)
    elif treebank_tag[1].startswith('N'):
        return (treebank_tag[0],wordnet.NOUN)
    elif treebank_tag[1].startswith('R'):
        return (treebank_tag[0],wordnet.ADV)
    else:
        (treebank_tag[0],wordnet.NOUN)

In [ ]:
 def get_pos_tag(list_of_tokens):
    return map(get_wnet_pos_tag,pos_tag(list_of_tokens))


In [ ]:
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk import pos_tag


In [ ]:
data['message to examine'] = data['message to examine'].apply(lambda x: get_pos_tag(x))
data['message to examine'] = data['message to examine'].apply(lambda x: list(x))

In [ ]:
data['message to examine']

In [ ]:
lst = []
for i in data['message to examine']:
    lst.append(len(i))
    
data['len'] = lst

data = data[data.len != 0]

data

In [ ]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [ ]:
def token_lemmatization(token_pos_tuple):
    if token_pos_tuple == None:
        return ""
    else:
        return lemmatizer.lemmatize(word=token_pos_tuple[0],pos=token_pos_tuple[1])

def lemmatization(list_of_tokens):
    if len(list_of_tokens) > 0:
        return map(lambda x: token_lemmatization(x),list_of_tokens)
        

In [ ]:
data['message to examine'] = data['message to examine'].apply(lambda x: lemmatization(x))
data['message to examine'] = data['message to examine'].apply(lambda x: list(x))
data['message to examine']

In [ ]:
vocab = set()
for list_f_tok in data['message to examine']:
    vocab = vocab.union(set(list_f_tok))

len(vocab)

In [ ]:
data.drop(['len'],axis=1,inplace=True)
data.head()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

def join_tokens(lst_of_token):
    return " ".join(lst_of_token)
data['message to examine'] = data['message to examine'].apply(lambda  x: join_tokens(x))

In [ ]:
corpus = []

for email in data['message to examine']:
    corpus.append(email)
    
    
tf_idf_matrix = vectorizer.fit_transform(corpus)
tf_idf_matrix.shape

In [ ]:
tf_idf_matrix = tf_idf_matrix.toarray()
train_data = pd.DataFrame(tf_idf_matrix)
train_data['target'] = data['label (depression result)']

In [ ]:
train_data.head()

In [ ]:
train_data.shape

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=6000)
data_pca = pca.fit_transform(train_data.iloc[:, :-1])
data_pca.shape

In [ ]:
df_pca_loadings = pd.DataFrame(pca.components_)
df_pca_loadings

In [ ]:
data_pca = pd.DataFrame(data_pca)
print(pca.explained_variance_ratio_.round(2)[:10])

In [ ]:
import matplotlib.pyplot as plt
variance_exp_cumsum = pca.explained_variance_ratio_.cumsum().round(2)
fig, axes = plt.subplots(1,1,figsize=(16,7), dpi=100)
plt.plot(variance_exp_cumsum, color='firebrick')
plt.title('Screeplot of Variance Explained %', fontsize=22)
plt.xlabel('# of PCs', fontsize=16)
plt.show()

In [ ]:
from scipy.spatial import ConvexHull

def encircle(x,y, ax=None, **kw):
    if not ax: ax=plt.gca()
    p = np.c_[x,y]
    hull = ConvexHull(p)
    poly = plt.Polygon(p[hull.vertices,:], **kw)
    ax.add_patch(poly)
    
rows_0 = data['label (depression result)']==0;
rows_1 = data['label (depression result)']==1; 

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(16,12))

# Plot
ax.scatter(data_pca.loc[rows_0.tolist(), 1], data_pca.loc[rows_0.tolist(), 2], c='blue', edgecolor='k', s=120, label='Not Spam')
ax.scatter(data_pca.loc[rows_1.tolist(), 1], data_pca.loc[rows_1.tolist(), 2], c='red', edgecolor='k', s=120, label='Spam')

# Encircle the boundaries
encircle(data_pca.loc[rows_0.tolist(), 1], data_pca.loc[rows_0.tolist(), 2], ec="blue", fc="none", linewidth=2.5)
encircle(data_pca.loc[rows_1.tolist(), 1], data_pca.loc[rows_1.tolist(), 2], ec="firebrick", fc="none", linewidth=2.5)

# Shading
encircle(data_pca.loc[rows_1.tolist(), 1], data_pca.loc[rows_1.tolist(), 2], ec="k", fc="firebrick", alpha=0.05)
encircle(data_pca.loc[rows_0.tolist(), 1], data_pca.loc[rows_0.tolist(), 2], ec="k", fc="blue", alpha=0.05)

# Labels
ax.set_title("Spam or Not spam data: Scatterplot of First Two PCA directions", fontsize=22)
ax.set_xlabel("1st Principal Component", fontsize=22)
ax.set_ylabel("2nd Principal Component", fontsize=22)
ax.legend(loc='best', title='Email Type', fontsize=16)
plt.show();

In [ ]:
data_pca['target'] = pd.Series(list(data['label (depression result)']), index=data_pca.index)
data_pca.target.unique()

In [ ]:
data_pca.head()

In [ ]:
import imblearn
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from collections import Counter
counter = Counter(data_pca.target)
print(counter)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_pca.iloc[:, :-1], data_pca.target, test_size=0.33, random_state=42)

In [ ]:
ctrain = Counter(y_train)
print(ctrain)

In [ ]:
ctrain = Counter(y_test)
print(ctrain)

In [ ]:
X_train.shape

In [ ]:
over = SMOTE(sampling_strategy=0.5)
under = RandomUnderSampler(sampling_strategy=0.7)
steps = [('o', over), ('u', under)]
pipeline = Pipeline(steps=steps)
# transform the dataset
X_train, y_train = pipeline.fit_resample(X_train, y_train)
# summarize the new class distribution
counter = Counter(y_train)
print(counter)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from scipy.stats import loguniform


In [ ]:
# define model
model = LogisticRegression()
# define evaluation
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# define search space
space = dict()
space['solver'] = ['newton-cg', 'lbfgs', 'liblinear']
space['penalty'] = ['none', 'l1', 'l2', 'elasticnet']
space['C'] = loguniform(1e-5, 100)
# define search
search = RandomizedSearchCV(model, space, n_iter=100, scoring='recall', n_jobs=-1, cv=cv, random_state=1,verbose=3)
# execute search
result = search.fit(X_train, y_train)
# summarize result
print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)

In [ ]:
from sklearn import metrics
from sklearn.metrics import classification_report,confusion_matrix

predicted_categories = result.predict(X_test)
print(classification_report(y_true=y_test,y_pred=predicted_categories))

In [ ]:
import seaborn as sns
plt.figure(figsize=(15,8))
conf_mx = confusion_matrix(y_test, predicted_categories)
conf_mx
sns.heatmap(conf_mx, annot=True, linewidths=.5, fmt= '.1f')

In [ ]:
plt.figure(figsize=(20,12))
metrics.plot_roc_curve(result, X_test, y_test)  
plt.show()